In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


*Sales* is our target variable
*TV*, *radio* and *newspaper* are our features
Our goal is to define *sales* as some sort of equation from the variables - TV, radio and newspaper

In [8]:
X = df.drop("sales", axis=1)
y = df["sales"]

Steps:
- Polynomial feature conversion
- Feature scaling using Standardization or Normalization
- Elastic net

In [6]:
from sklearn.preprocessing import PolynomialFeatures

In [7]:
poly_converter = PolynomialFeatures(degree=3, include_bias=False)

How did we know that 3rd degree polynomial would give the best results?

In [9]:
poly_features = poly_converter.fit_transform(X)

In [11]:
type(poly_features)

numpy.ndarray

In [13]:
poly_features[0]

array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
       8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
       4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
       3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
       9.88757280e+04, 1.81010592e+05, 3.31373888e+05])

In [14]:
poly_features.shape

(200, 19)

In [15]:
## Splitting poly_features into training and testing data
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
scaler = StandardScaler()

In [17]:
# We will fit the scaler on training data, not on the testing data
scaler.fit(X_train)

StandardScaler()

In [18]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
X_train.shape

(140, 19)

In [20]:
X_train[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [21]:
# Now we will apply Elastic Net regression
from sklearn.linear_model import ElasticNetCV

In [23]:
elastic_cv_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1], eps=0.001, max_iter=1000000)

In [24]:
elastic_cv_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [25]:
elastic_cv_model.l1_ratio_

1.0

As we can infer from the output l1_ratio of the elastic net regression, its suggesting to go ahead with l1 regularization (also called lasso regression) and completely ignore l2 regularization (also known as ridge regression)

In [29]:
elastic_cv_model.coef_

array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

As we can see above, quite a few coefficients are zero and the regression equation is a relatively simple one

In [30]:
# Predicting on the testing dataset
# X_test is our hold out test dataset and whatever test results we receive, it would be only used for reporting 
# and not for re-training our model
test_predictions = elastic_cv_model.predict(X_test)

In [31]:
test_predictions.shape

(60,)

In [32]:
#Performing our test metrics (e.g MAE, RMSE)
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [33]:
Elastic_model_MAE = mean_absolute_error(y_test, test_predictions)
Elastic_model_MAE

0.43350346185900707

In [34]:
Elastic_model_RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))
Elastic_model_RMSE

0.6063140748984027

As we can see from the above metrics is that our model has performed quite well, given the fact that its a relatively simple model